# The Gladius Data Model

### Data Model Structure


Collection: "clubs"

Document: "club_id"
Club data fields
Sub-collection: "events"
Document: "event_id"
Event data fields
Sub-collection: "occurrences"
Document: "occurrence_id"
Occurrence data fields
Sub-collection: "attendance"
Document: "attendance_id"
Field: "user_id" (to reference the attending user)
Field: "score" (record the score for the user at this occurrence)

Collection: "users"

Document: "user_id"
User data fields
Sub-collection: "scores"
Document: "score_id"
Field: "club_id" (to reference the club)
Field: "event_id" (to reference the event)
Field: "occurrence_id" (to reference the occurrence)
Field: "score" (the user's score for this event occurrence)


In [14]:
# Installs
#!pip install firebase-admin

In [15]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore
import random
from dateutil.rrule import rrulestr
from dateutil.parser import parse
import datetime
import os
import firebase_admin
from firebase_admin import credentials, firestore

os.environ["GCLOUD_PROJECT"] = "wallet-login-45c1c"

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

Authentication

In [16]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Create users with authentication

### Delete all auth users

In [17]:
user_list = auth.list_users()

# Iterate over each user and delete them
for user in user_list.users:
    auth.delete_user(user.uid)
    print(f"Deleted user: {user.uid}")

print("All users have been deleted.")

Deleted user: 6uuGNDujf3faRDxPldlMSl9Wnns2
Deleted user: GYHcWELQkFMpd9NidY8fyv3zER43
Deleted user: OYsTIWxITJTDRrMRX3FOZ4zeX3F2
Deleted user: WA5Gk2q1sZduXH3BNDNWVHJw8Ei2
Deleted user: ZmJ8Mb0bEoZUqOfm8SH8XKjsNrP2
Deleted user: f0bOQGOlLAQp4gzjJexbSaKoRp13
Deleted user: k4s5ZsOWF6SV8h3Gh4aSnCBelWj2
Deleted user: mxFfl1jym8NWkcxjxybuXzsK5Xp1
Deleted user: oq7zFTYUdqczFw7tsM5V4SXYYkU2
Deleted user: rsuqCcPclYP7RNApjFSonxzhGal1
Deleted user: uug2mnETM0PrZntIrf6nkw2JAPb2
All users have been deleted.


### Create new auth users

In [18]:
# Create a list of users to add
users = ["Mare", "Jüri", "Liisa", "Mart", "Evelin", "Tõnis", "Mari", "Henrik", "Triin", "Jaak", 'Bob']


user_uuids = {}
# Loop through the users and create them with a default password
for user in users:
  try:
    # Create the user with the email and password
    user_record = auth.create_user(
      email=f'{user.lower()}@example.com',
      password='123456'
    )
    user_uuids[user] = user_record.uid
    # Print the user ID
    print(f'Successfully created new user: {user_record.uid}')
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error creating new user: {e}')

Successfully created new user: JVeis00f4OZOrsClkfju3eHWkDw2
Successfully created new user: IgfIPWMCvYfzRbYtx8LVTzYBLa03
Successfully created new user: gqXaSmsSLFY2u5TEqdLLQuT1sOy2
Successfully created new user: jpQn2ga9wrM08dgPUh0O6HE98tB2
Successfully created new user: xrbk7RSnD8ehmxzKohYjkfj6i1n1
Successfully created new user: 7gDfM8iz83V0FuPiwpPcz5DK2C52
Successfully created new user: ccz94n2wSRWRFAlvQ4eZoUpAkDR2
Successfully created new user: 8BAvxpwSAlOtS9hCsy7mrzwKLUu2
Successfully created new user: DluEce1EAjYOFlWwdqZlA86LoQ32
Successfully created new user: HR09T0mSawhHW9MqfLaDnfI5hdz1
Successfully created new user: xwco0L2NAbddFcfmXQAhgomnX1A2


### Delete user collection

In [19]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("users")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Users collection deleted successfully.")

Users collection deleted successfully.


## Add users collection

In [20]:
# Create a collection for users
users_collection = db.collection("users")

user_clubs_roles = {}

for user in users:
    user_data = {
        "name": user,
        "email": f"{user.lower()}@example.com",
        "age": random.randint(18, 60),
        "occupation": random.choice(["Student", "Engineer", "Teacher", "Artist"]),
        "is_active": random.choice([True, False]),
        "clubs_roles": [],  # initialize an empty list for clubs and roles
    }

    user_ref = users_collection.document(user_uuids[user])
    user_ref.set(user_data)
    print(f"Successfully created user: {user_uuids[user]}")

    if user in ["Bob", "Mare"]:
        user_clubs_roles[user_uuids[user]] = [{"club_id": "1", "role": "owner"}]
    else:
        user_clubs_roles[user_uuids[user]] = [{"club_id": "1", "role": "athlete"}]

# After creating all users, update the users with their clubs and roles
for user_uuid, clubs_roles in user_clubs_roles.items():
    user_ref = users_collection.document(user_uuid)
    user_ref.update({"clubs_roles": clubs_roles})


Successfully created user: JVeis00f4OZOrsClkfju3eHWkDw2
Successfully created user: IgfIPWMCvYfzRbYtx8LVTzYBLa03
Successfully created user: gqXaSmsSLFY2u5TEqdLLQuT1sOy2
Successfully created user: jpQn2ga9wrM08dgPUh0O6HE98tB2
Successfully created user: xrbk7RSnD8ehmxzKohYjkfj6i1n1
Successfully created user: 7gDfM8iz83V0FuPiwpPcz5DK2C52
Successfully created user: ccz94n2wSRWRFAlvQ4eZoUpAkDR2
Successfully created user: 8BAvxpwSAlOtS9hCsy7mrzwKLUu2
Successfully created user: DluEce1EAjYOFlWwdqZlA86LoQ32
Successfully created user: HR09T0mSawhHW9MqfLaDnfI5hdz1
Successfully created user: xwco0L2NAbddFcfmXQAhgomnX1A2


## Delete Club collection

In [29]:
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        
        # Delete subcollections of the document
        for subcoll in doc.collections():
            delete_collection(subcoll, batch_size)
            
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

# Specify the "clubs" collection to be deleted
clubs_collection_ref = db.collection("clubs")

# Delete the "clubs" collection and its documents
delete_collection(clubs_collection_ref, 500)


Deleting doc 1 => {'description': 'A Learn to Earn Club', 'calendars': ['dcromp88@googlemail.com'], 'name': 'Gladius Club'}
Deleting doc dcCSSnP3nCY7fNcgwQXS => {'event_ids': [{'calendarId': 'dcromp88@googlemail.com', 'eventId': '4d06bu8godice3gdt7qs8k3cg4'}], 'member_uuids': ['HR09T0mSawhHW9MqfLaDnfI5hdz1', 'IgfIPWMCvYfzRbYtx8LVTzYBLa03', 'DluEce1EAjYOFlWwdqZlA86LoQ32', 'gqXaSmsSLFY2u5TEqdLLQuT1sOy2', 'xrbk7RSnD8ehmxzKohYjkfj6i1n1', '7gDfM8iz83V0FuPiwpPcz5DK2C52', 'xwco0L2NAbddFcfmXQAhgomnX1A2', '8BAvxpwSAlOtS9hCsy7mrzwKLUu2', 'jpQn2ga9wrM08dgPUh0O6HE98tB2', 'ccz94n2wSRWRFAlvQ4eZoUpAkDR2'], 'name': 'Random Group'}
Deleting doc 7gDfM8iz83V0FuPiwpPcz5DK2C52 => {'name': 'Tõnis', 'user': '7gDfM8iz83V0FuPiwpPcz5DK2C52', 'email': 'tõnis@example.com', 'role': 'athlete'}
Deleting doc dcromp88@googlemail.com_4d06bu8godice3gdt7qs8k3cg4_20230715T110000Z => {'win': None, 'calendarId': 'dcromp88@googlemail.com', 'eventParentId': '4d06bu8godice3gdt7qs8k3cg4', 'eventId': '4d06bu8godice3gdt7qs8k3cg4_

## Add club collection

In [49]:

# Create a collection for clubs
clubs_collection = db.collection("clubs")

# Create a document for a club with some data fields
club_id = '1'
club_data = {
    "name": "Gladius Club",
    "description": "A Learn to Earn Club",
    "calendars": ["dcromp88@googlemail.com"],
}
club_ref = clubs_collection.document(club_id)
club_ref.set(club_data)

# Create a users subcollection under the club
users_subcollection = club_ref.collection("members")

# Add users to the users subcollection
for user_uuid, clubs_roles in user_clubs_roles.items():
    for role in clubs_roles:
        if role["club_id"] == "1":
            # Get user data from the users collection
            user_doc_ref = users_collection.document(user_uuid)
            user_snapshot = user_doc_ref.get()
            user_data = user_snapshot.to_dict()

            # Create the user data to be stored in the subcollection
            user_subcollection_data = {
                "user": user_uuid,
                "name": user_data["name"],
                "email": user_data["email"],
                "role": role["role"],
            }
            users_subcollection.document(user_uuid).set(user_subcollection_data)



### Create groups

In [50]:

# The ID of the club under which the group will be created
club_id = "1"
club_ref = clubs_collection.document(club_id)

# Get all user UUIDs from the users collection
user_uuids = [doc.id for doc in users_collection.stream()]

# Select a random subset of users to be members of the group
member_uuids = random.sample(user_uuids, 10)  # choose 10 members randomly

group_data = {
    "name": "Random Group",  # the name of the group
    "member_uuids": member_uuids,  # the UUIDs of the group's members
    "event_ids": [  # list of events, where each event is an object with a calendarId and eventId
        {
            "calendarId": "dcromp88@googlemail.com",  # replace with actual calendarId
            "eventId": "4d06bu8godice3gdt7qs8k3cg4"  # replace with actual eventId
        },
        # more events can go here...
    ],
}

# Create the group in the groups subcollection under the club
groups_subcollection = club_ref.collection("groups")
group_ref = groups_subcollection.document()  # generate a random ID for the group
group_ref.set(group_data)

print(f"Successfully created group: {group_ref.id} under club: {club_id}")


Successfully created group: d4DZstYsnf8vyOhYrrrS under club: 1


## Add events to members

In [51]:
googleCalendarId = "dcromp88@googlemail.com"


SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'

In [47]:
def import_events():
    # List to store events
    events_list = []

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    print('Getting the upcoming 20 events')
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                          maxResults=100, singleEvents=False).execute()
    events = events_result.get('items', [])

    for event in events:
        if 'recurrence' in event:  # if the event is a recurring event
            # Getting individual instances of the recurring event
            instances_result = service.events().instances(calendarId=googleCalendarId, eventId=event['id']).execute()
            instances = instances_result.get('items', [])
            events_list.extend(instances)  # Add the instances to the event list
        else:
            events_list.append(event)  # Add the non-recurring event to the event list

        
    events_list = sorted(events_list, key=lambda k: parse(k['start']['dateTime'] if 'dateTime' in k['start'] else k['start']['date'] + 'T00:00:00Z'))

    return events_list


events = import_events()


Getting the upcoming 20 events


In [52]:

# Get the members subcollection under the club
members_subcollection = club_ref.collection("members")

# Specify calendarId and eventId
calendarId = "dcromp88@googlemail.com"

# Constants
MIN_SCORE = 0  # Define your minimum score here
MAX_SCORE = 100  # Define your maximum score here

for event in events[0:100]:
    eventId = event['id']  # get eventId from the current event
    # Get the start time of the event
    event_start = event['start'].get('dateTime', event['start'].get('date'))
    # Parse the start time to a datetime object
    event_date = parse(event_start)
    # Format the date to "yyyy-mm-dd"
    event_date_str = event_date.strftime('%Y-%m-%d')

    
    # Add attendance records to the attendance subcollection of each member
    for user_uuid in member_uuids:
        # Get the document reference for the member
        member_doc_ref = members_subcollection.document(user_uuid)
        
        # Create an attendance subcollection under the member
        attendance_subcollection = member_doc_ref.collection("attendance")

        # 90% chance they attended
        attended = random.choices([True, False], weights=[90, 10], k=1)[0]

        win = random.choices([True, False], weights=[50, 50], k=1)[0]

        if attended:
            # Prepare attendance data
            attendance_data = {
                "attended": attended,
                "calendarId": calendarId,
                "coefficient": None,
                "date": event_date_str,
                "eventId": eventId,
                "eventParentId": eventId.split('_')[0],  # splits eventId on '_' and gets the first part
                "score": random.randint(MIN_SCORE, MAX_SCORE) if attended else None,  # generate a random score if attended
                "win": win if attended else None,
            }

            # generate a document ID from "calendarId_eventId"
            document_id = f"{calendarId}_{eventId}"
            attendance_ref = attendance_subcollection.document(document_id)  
            attendance_ref.set(attendance_data)


### Add events to Firebase [events] collection

In [36]:



def get_next_occurrence(rrule_str, dtstart):
    rrule = rrulestr(rrule_str.replace('RRULE:', ''), dtstart=dtstart)
    next_occurrence = rrule.after(datetime.datetime.now(datetime.timezone.utc))
    return next_occurrence.isoformat()

def import_events():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                          maxResults=10, singleEvents=False).execute()
    events = events_result.get('items', [])

    if not firebase_admin._apps:
        firebase_cred = credentials.Certificate("../firebase-adminsdk.json")
        default_app = firebase_admin.initialize_app(firebase_cred)

    db = firestore.client()

    for event in events:
        is_recurring = 'recurrence' in event
        event_id = event['id']
        summary = event.get('summary', '')
        location = event.get('location', '')
        recurrence_rules = event.get('recurrence', []) if is_recurring else []
        start = event['start'].get('dateTime', event['start'].get('date'))

        next_occurrence = None
        if is_recurring and recurrence_rules:
            next_occurrence = get_next_occurrence(recurrence_rules[0], parse(start))
        else:
            next_occurrence = start

        event_info = {
            'id': event_id,
            'is_recurring': is_recurring,
            'recurrence_rules': recurrence_rules,
            'summary': summary,
            'location': location,
            'start': start,
            'next_occurrence': next_occurrence,
        }

        event_ref = db.collection('events').document(event_id)
        event_ref.set(event_info)
        print(event_info, ' added')


if __name__ == '__main__':
    import_events()


Getting the upcoming 10 events
{'id': '4d06bu8godice3gdt7qs8k3cg4', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=MONTHLY;BYDAY=3SA'], 'summary': 'Competition ', 'location': '', 'start': '2023-05-20T13:00:00+02:00', 'next_occurrence': '2023-07-15T13:00:00+02:00'}  added
{'id': '_8d136ea184o42ba36l1jeb9k8kqjaba264qjgba36t2jgdhp8gq44ha66s', 'is_recurring': False, 'recurrence_rules': [], 'summary': 'Fathers day', 'location': '', 'start': '2023-11-12', 'next_occurrence': '2023-11-12'}  added
{'id': '2ai848el5g91a93m61cvc9uesm', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;WKST=SU;BYDAY=FR,TU'], 'summary': 'Training', 'location': '', 'start': '2023-05-16T19:00:00+02:00', 'next_occurrence': '2023-07-04T19:00:00+02:00'}  added
{'id': '6cm8ofu930v8ijkkomtf6ktjdf', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;BYDAY=FR'], 'summary': 'Adult Training', 'location': '', 'start': '2023-05-19T20:00:00+02:00', 'next_occurrence': '2023-07-07T20:00:00+02:00'}  a

In [5]:
firestore_client = firestore.client()
# Create a collection named 'groups'
groups_ref = firestore_client.collection('groups')
# Create a document with ID 'group1' and some data
group1_ref = groups_ref.document('group1')
group1_ref.set({
    'name': 'Group 1',
    'members': ['Alice', 'Bob', 'Charlie']
})

# Create another document with ID 'group2' and some data
group2_ref = groups_ref.document('group2')
group2_ref.set({
    'name': 'Group 2',
    'members': ['David', 'Eve', 'Frank']
})